In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

# Competition

Overview를 통해서 이 대회를 여는 이유와 근본적인 문제가 무엇인지 파악을 확실하게 하고, Domain의 대한 이해까지 함께 진행하자.

그리고 난 뒤에 EDA를 하는 방식으로 진행!

## 2강. 이미지 분류 & EDA

이미지에서 EDA는 무엇인가.

### 2-1. EDA
데이터를 이해하기 위한 노력, 왜 이런 데이터를 줬는지? 그리고 무엇인지 보는 과정

overview를 보고 내가 생각한 **설들을 데이터를 통해서 한번 확인해보자.**

EDA 각각의 셀들에 대한 이해를 진행하자.

* 목적없이 무작정 나오는 코드는 없고 그렇게 만들지도 말자!

### 2-2. 이미지

* uint8 : 0~255, unsigned특징으로 음수를 표현하지 못한다. 

모델의 목적과 데이터의형태에 따라 input과 output의 형태들이 모두 다르다. output의 형태에 따라서 과제가 달라진다.

# 3강. Dataset


주어진 Vanilla Data(Raw data)를 모델이 좋아하는, 모델에 넣을 수 있는 형태의 Dataset으로 구성해줘야 한다. 

### Preprocessing

* Bounding Box
* Resize

### Generalization

* Bias & Variance
* Train / Validation
* Data Augmentation : 주어진 데이터가 가질 수 있는 Case(경우)와 State(상태)의 다양성을 준다.
    * 원본, 밤, 폭우, 폭설.. 등 이 도메인에서 고려할 수 있는 상황을 모두 담은 데이터 셋을 만들어 보면 좋지 않을까?
    * 이 데이터에서 실제로 발생할 수 있는 여러 상황들을 정의하는 것이 중요
    
* Albumentations : torchvision외의 Augmentation tools
    

# 4강. Data Generalization
모델 학습에 있어서 데이터가 모델 학습에 유용한 형태로 빠르게 Generation을 할 수 있어야 한다.


### 4-1. Data Feeding

설계 - 제작 - 포장 과 같이, 데이터를 만들어 모델에 넣어주고, 데이터를 받은 모델이 연산하는 과정을 서로 균일하게 맞춰줘야 효과가 있다.

* ex) Generator(10 batch/s) -> Model(20 batch/s) = Max 10b/s
* ex) Generator(30 batch/s) -> Model(20 batch/s) = Max 20b/s


### 4-2. torch.utils.data


#### Dataset
Vanlia data를 학습에 용이하게 전처리를 해주는 파트라고 생각하자. 여기서 효율적으로 작업이 되어 Dataloader에게 잘 넘어가게 해줘야 한다.

In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        pass
    
    def __getitem__(self):
        pass
    
    def __len__(self):
        return None

#### DataLoader

Dataloader는 Dataset과는 달리, 제대로 된것 한개만 만들어서 사용하면 된다. 

# 5강. Model - with Pytorch


In [1]:
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

In [3]:
a = MyModel()
print(a)

MyModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)


In [4]:
# modules() : 제네레이터를 만들면서 보여줌
list(a.modules())

[MyModel(
   (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
   (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
 ),
 Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1)),
 Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))]

In [5]:
a.state_dict() # key 값과 paramater가 함께

OrderedDict([('conv1.weight',
              tensor([[[[ 0.0657, -0.1845,  0.0679,  0.0771,  0.1874],
                        [-0.0188, -0.1657,  0.0175, -0.1652, -0.1149],
                        [-0.0817,  0.1956, -0.1411, -0.0014, -0.0935],
                        [-0.1437, -0.0890, -0.0115, -0.0335,  0.1911],
                        [ 0.1569,  0.1313,  0.0565, -0.1578, -0.1804]]],
              
              
                      [[[-0.1045, -0.1460,  0.1763,  0.0471, -0.0205],
                        [-0.1122,  0.1793, -0.0136, -0.1610, -0.0652],
                        [ 0.0422,  0.0576, -0.0240, -0.0569,  0.1994],
                        [ 0.1676,  0.0163,  0.0320, -0.0172,  0.1386],
                        [ 0.1773,  0.1913,  0.0669, -0.1143, -0.0532]]],
              
              
                      [[[-0.1518, -0.0846, -0.1273,  0.1311, -0.1138],
                        [ 0.0273, -0.1984,  0.1787,  0.1284, -0.1615],
                        [ 0.0397, -0.1375, -0.1138, -0

In [6]:
list(a.parameters()) # paramater만

[Parameter containing:
 tensor([[[[ 0.0657, -0.1845,  0.0679,  0.0771,  0.1874],
           [-0.0188, -0.1657,  0.0175, -0.1652, -0.1149],
           [-0.0817,  0.1956, -0.1411, -0.0014, -0.0935],
           [-0.1437, -0.0890, -0.0115, -0.0335,  0.1911],
           [ 0.1569,  0.1313,  0.0565, -0.1578, -0.1804]]],
 
 
         [[[-0.1045, -0.1460,  0.1763,  0.0471, -0.0205],
           [-0.1122,  0.1793, -0.0136, -0.1610, -0.0652],
           [ 0.0422,  0.0576, -0.0240, -0.0569,  0.1994],
           [ 0.1676,  0.0163,  0.0320, -0.0172,  0.1386],
           [ 0.1773,  0.1913,  0.0669, -0.1143, -0.0532]]],
 
 
         [[[-0.1518, -0.0846, -0.1273,  0.1311, -0.1138],
           [ 0.0273, -0.1984,  0.1787,  0.1284, -0.1615],
           [ 0.0397, -0.1375, -0.1138, -0.0652, -0.0696],
           [-0.0752, -0.1854,  0.1819, -0.1555, -0.1161],
           [ 0.1676,  0.0504, -0.1032, -0.1606, -0.0799]]],
 
 
         [[[ 0.1729,  0.1938, -0.0791, -0.1541, -0.1090],
           [-0.0616,  0.0891, -

각 모델의 파라미터들은 data, grad, requires_grad 변수 등을 갖고 있다.

* requires_grad : 자동 미분 여부
* grad : 자동 미분의 결과값, 누적이 되는 형식

In [7]:
for param in a.parameters():
    print(f"required_grad : {param.requires_grad}")
    print(f"grad : {param.grad}")

required_grad : True
grad : None
required_grad : True
grad : None
required_grad : True
grad : None
required_grad : True
grad : None
